## Dependencies:
Pandas: `pip install pandas`<br>
Pillow: `pip install pillow`<br>
xlrd: `conda install -c conda-forge openpyxl`<br>
tqdm: `pip install tqdm`<br>

***Only for development environment***<br>
Jupyter Notebook: `conda install -c conda-forge notebook`<br>
Jupyter Notebook Conda Connectors: `conda install -c conda-forge nb_conda_kernels`<br>

In [1]:
import os
import json
import time
import logging
import pandas as pd

from tqdm import tqdm
from datetime import datetime, timedelta
from PIL import Image, ImageFont, ImageDraw

## Logging

In [2]:
if not os.path.exists(r"logs/"):
    logging.debug("No directory for logs found. Creating logs directory.")
    os.makedirs(r"logs/")

# Set up logging to file
LOGGING_DIR = r'logs/'
session_startTime = datetime.now()
local_tzname = (((session_startTime).astimezone()).tzinfo).tzname((session_startTime).astimezone())
logfile = session_startTime.strftime("Session_%d%b%y_%H%M%S.log")
# NOTE: File logging has been limited to INFO level
logging.basicConfig(format='%(name)s %(asctime)s: %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S', filename=LOGGING_DIR+logfile)

# Set up logging for display stdout
console = logging.StreamHandler()
# NOTE: File logging has been not restricted
console.setLevel(logging.NOTSET)
formatter = logging.Formatter('%(asctime)s: %(message)s')
console.setFormatter(formatter)
logging.getLogger().addHandler(console)

## Settings

In [3]:
SETTINGS_FILE = "settings.json"

with open(SETTINGS_FILE, mode='r', encoding='utf-8-sig') as f:
    settings = json.load(f)

In [4]:
def read_excel_file(file_details):
    
    # Read excel file
    df = pd.read_excel(file_details['xlsx_file'], engine='openpyxl', skiprows=file_details['skip_rows'])
    
    # Strip whitespace in str columns
    for col in df.columns:
        if df.dtypes[col]=='O':
            df[col].str.strip()
            df.fillna(value={col: ""})
    
    # Set primary key
    df.set_index(file_details['primary_key'], inplace=True)
    
    return df

In [5]:
item_catalog = read_excel_file(settings['item_catalog'])

In [6]:
purchase_orders = read_excel_file(settings['purchase_orders'])

In [7]:
# Sorting items according to purchase order dates
item_catalog['last_purchase_date'] = datetime(year=1990, month=1, day=1)
item_catalog['last_purchase'] = ''

for i in list(item_catalog.index):
    try:
        last_purchase_date = max(purchase_orders.at[i, 'Date'])
    except KeyError as e:
        item_catalog.at[i, 'last_purchase'] = 'Others'
        continue
    except TypeError as e:
        last_purchase_date = purchase_orders.at[i, 'Date']
    item_catalog.at[i, 'last_purchase_date'] = last_purchase_date
    if last_purchase_date>=(datetime.now()-timedelta(weeks=4)):
        item_catalog.at[i, 'last_purchase'] = 'This Month'
    elif last_purchase_date>=(datetime.now()-timedelta(weeks=8)):
        item_catalog.at[i, 'last_purchase'] = 'Last Month'
    else:
        item_catalog.at[i, 'last_purchase'] = 'Others'

In [8]:
# Reading JPG file paths
item_catalog['ip_file_path'] = ''
filepaths = os.listdir(settings['img_input_folder'])

for i in list(item_catalog.index):
    if ((i+".jpg") in filepaths):
        item_catalog.at[i, 'ip_file_path'] = settings['img_input_folder']+r"/"+i+".jpg"
    elif ((i+".jpeg") in filepaths):
        item_catalog.at[i, 'ip_file_path'] = settings['img_input_folder']+r"/"+i+".jpeg"
    else:
        logging.warning(f"Image not found for item {i}")
        continue

2022-09-14 23:39:23,596: Image not found for item SJ-SBN-1001L
2022-09-14 23:39:23,602: Image not found for item SJ-SBN-1002L
2022-09-14 23:39:23,605: Image not found for item SJ-SBN-21L97
2022-09-14 23:39:23,608: Image not found for item SJ-SBN-22A31
2022-09-14 23:39:23,611: Image not found for item SJ-SBN-22A32
2022-09-14 23:39:23,613: Image not found for item SJ-SBN-414
2022-09-14 23:39:23,615: Image not found for item SJ-SBN-4174
2022-09-14 23:39:23,616: Image not found for item SJ-SBN-8509
2022-09-14 23:39:23,617: Image not found for item SJ-SBN-8510
2022-09-14 23:39:23,619: Image not found for item SJ-SBN-8526
2022-09-14 23:39:23,621: Image not found for item SJ-SBN-9155
2022-09-14 23:39:23,622: Image not found for item SJ-SBN-9156
2022-09-14 23:39:23,623: Image not found for item SJ-SBN-9191
2022-09-14 23:39:23,625: Image not found for item SJ-SSP-056
2022-09-14 23:39:23,629: Image not found for item SJ-SSP-105
2022-09-14 23:39:23,630: Image not found for item SJ-SSP-132L
2022-0

2022-09-14 23:39:23,809: Image not found for item SJ-SHB-22F20
2022-09-14 23:39:23,810: Image not found for item SJ-SHB-22F21
2022-09-14 23:39:23,811: Image not found for item SJ-SHB-22F24
2022-09-14 23:39:23,813: Image not found for item SJ-SHB-22F25
2022-09-14 23:39:23,814: Image not found for item SJ-SHB-323
2022-09-14 23:39:23,815: Image not found for item SJ-SHB-324
2022-09-14 23:39:23,816: Image not found for item SJ-SHB-3630N
2022-09-14 23:39:23,817: Image not found for item SJ-SHB-3632N
2022-09-14 23:39:23,819: Image not found for item SJ-SHB-479
2022-09-14 23:39:23,820: Image not found for item SJ-SHB-6090
2022-09-14 23:39:23,821: Image not found for item SJ-SHB-6095
2022-09-14 23:39:23,822: Image not found for item SJ-SHB-6832
2022-09-14 23:39:23,823: Image not found for item SJ-SHB-6967
2022-09-14 23:39:23,825: Image not found for item SJ-SHB-7070
2022-09-14 23:39:23,828: Image not found for item SJ-SHB-8002
2022-09-14 23:39:23,832: Image not found for item SJ-SHB-8009
2022-

2022-09-14 23:39:24,008: Image not found for item SJ-SJP-FANCY
2022-09-14 23:39:24,009: Image not found for item SJ-SRB-098
2022-09-14 23:39:24,011: Image not found for item SJ-SRB-099
2022-09-14 23:39:24,012: Image not found for item SJ-SRB-101
2022-09-14 23:39:24,014: Image not found for item SJ-SRB-102
2022-09-14 23:39:24,017: Image not found for item SJ-SRB-107N
2022-09-14 23:39:24,018: Image not found for item SJ-SRB-106N
2022-09-14 23:39:24,019: Image not found for item SJ-SRB-105N
2022-09-14 23:39:24,020: Image not found for item SJ-SRB-104N
2022-09-14 23:39:24,021: Image not found for item SJ-SRB-107
2022-09-14 23:39:24,023: Image not found for item SJ-SRB-108
2022-09-14 23:39:24,024: Image not found for item SJ-SRB-1101L
2022-09-14 23:39:24,025: Image not found for item SJ-SRB-1102L
2022-09-14 23:39:24,026: Image not found for item SJ-SRB-1103L
2022-09-14 23:39:24,028: Image not found for item SJ-SRB-1115
2022-09-14 23:39:24,029: Image not found for item SJ-SRB-1245
2022-09-14

2022-09-14 23:39:24,199: Image not found for item SJ-STT-434
2022-09-14 23:39:24,200: Image not found for item SJ-STT-542
2022-09-14 23:39:24,201: Image not found for item SJ-STT-543
2022-09-14 23:39:24,202: Image not found for item SJ-STT-589
2022-09-14 23:39:24,203: Image not found for item SJ-STT-591
2022-09-14 23:39:24,204: Image not found for item SJ-STT-9067
2022-09-14 23:39:24,205: Image not found for item SJ-STT-9177
2022-09-14 23:39:24,207: Image not found for item SJ-STT-9306
2022-09-14 23:39:24,208: Image not found for item SJ-STT-9307
2022-09-14 23:39:24,210: Image not found for item SJ-STT-9501
2022-09-14 23:39:24,211: Image not found for item SJ-STT-9505
2022-09-14 23:39:24,212: Image not found for item SJ-STT-9506
2022-09-14 23:39:24,213: Image not found for item SJ-HBN-30102C
2022-09-14 23:39:24,215: Image not found for item SJ-HBN-30103B
2022-09-14 23:39:24,217: Image not found for item SJ-HBN-30105B
2022-09-14 23:39:24,218: Image not found for item SJ-HBN-301A
2022-09

2022-09-14 23:39:24,393: Image not found for item SJ-HCH-134502C
2022-09-14 23:39:24,395: Image not found for item SJ-HCH-134503B
2022-09-14 23:39:24,397: Image not found for item SJ-HCH-134505B
2022-09-14 23:39:24,398: Image not found for item SJ-HCH-134602C
2022-09-14 23:39:24,400: Image not found for item SJ-HCH-134603B
2022-09-14 23:39:24,402: Image not found for item SJ-HCH-134605B
2022-09-14 23:39:24,403: Image not found for item SJ-HCH-140102C
2022-09-14 23:39:24,405: Image not found for item SJ-HCH-140103B
2022-09-14 23:39:24,408: Image not found for item SJ-HCH-140105B
2022-09-14 23:39:24,409: Image not found for item SJ-HCH-131503G
2022-09-14 23:39:24,411: Image not found for item SJ-HCH-132103G
2022-09-14 23:39:24,412: Image not found for item SJ-HCH-134503G
2022-09-14 23:39:24,413: Image not found for item SJ-HCH-134603G
2022-09-14 23:39:24,414: Image not found for item SJ-HCH-140103G
2022-09-14 23:39:24,416: Image not found for item SJ-HCH-135602C
2022-09-14 23:39:24,417: 

In [16]:
def create_captioned_img(df, primary_key, labels, cat=None):
    
    global settings
    
    pipeline = list()
    total_height = 0
    
    raw_image = Image.open(df.at[primary_key, 'ip_file_path'])
    
    # Resizing image
    w,h = raw_image.size

    IMG_WIDTH = w
    new_height = h
    
    # new_height = int((h/w)*IMG_WIDTH)

#     logging.debug(f"Resizing image to {IMG_WIDTH}px x {new_height}px.")
#     raw_image = raw_image.resize((IMG_WIDTH, new_height))
    pipeline.append(raw_image)
    total_height += new_height
    
    font_size = int((w/2100)*settings['labelling_schema']['font_size'])
    FONT_SETTINGS = ImageFont.truetype(settings['labelling_schema']['font_file'], font_size)
    LINE_HEIGHT = int((w/2100)*settings['labelling_schema']['line_height']) # pixels
    LABELS_PER_LINE = settings['labelling_schema']['labels_per_line']
    
    
    # Blank line above
    im = Image.new(mode="RGB",size=(IMG_WIDTH, int(LINE_HEIGHT/2)), color=(250, 250, 250))
    pipeline.append(im)
    total_height += int(LINE_HEIGHT/2)
    
    # Preparing text to write
    txt_lines = [[]]
    for i in range(len(labels)):
        
        if len(txt_lines[-1])==LABELS_PER_LINE:
            txt_lines.append(list())
        
        if labels[i]==df.index.name:
            msg = labels[i]+": "+primary_key
        elif labels[i]=="Rate/Unit":
            rate_per_unit = f"{df.at[primary_key, cat]:.2f}/"+str(df.at[primary_key, 'Base Unit']) 
            msg = "Rate: ₹ "+rate_per_unit
        elif labels[i]=="Min Qty/Ord":
            msg = f"Min Ord: {df.at[primary_key, 'Min Qty']} {df.at[primary_key, 'Ord Unit']}" 
        else:
            msg = labels[i]+": "+str(df.at[primary_key, labels[i]])
        
        txt_lines[-1].append(msg)
    
    # Writing text to image line-by-line
    for lines in txt_lines:
        im = Image.new(mode="RGB",size=(IMG_WIDTH, LINE_HEIGHT), color=(250, 250, 250))
        draw = ImageDraw.Draw(im)
        msg = "        ".join(lines)
        draw.text((IMG_WIDTH/2, LINE_HEIGHT/2), msg, fill='black', font=FONT_SETTINGS, anchor='mm')
        pipeline.append(im)
        total_height += LINE_HEIGHT
    
    # Blank line below
    im = Image.new(mode="RGB",size=(IMG_WIDTH, int(LINE_HEIGHT/2)), color=(250, 250, 250))
    pipeline.append(im)
    total_height += int(LINE_HEIGHT/2)
    
    # Final Image
    final_img = Image.new(mode='RGB', size=(IMG_WIDTH, total_height))
    
    height_counter = 0
    for i in range(len(pipeline)):
        final_img.paste(pipeline[i], box=(0, height_counter))
        _, h = pipeline[i].size
        height_counter += h
    
    return final_img

In [17]:
OUTPUT_FOLDER = settings["img_output_folder"]

for i in tqdm(list(item_catalog[item_catalog['ip_file_path']!=''].index), desc='Creating images: ', unit='item'):
    if item_catalog.at[i, 'ip_file_path']!='':
        for variation in settings['labelling_schema']['label_variations']:

            img = create_captioned_img(item_catalog, i, variation['label_cols'], cat=variation['price_col'])
            
            if item_catalog.at[i, settings['labelling_schema']["to_clear_col_name"]] in ["Y", "y"]:
                heading = settings['labelling_schema']["to_clear_col_name"]
            else:
                heading = item_catalog.at[i, 'last_purchase']
            
            output_path = OUTPUT_FOLDER + r'/' + variation['price_col'] + r'/' + heading + r'/' + item_catalog.at[i, 'Group'] + r'/' + item_catalog.at[i, 'Category'] + r'/'
            
            if not os.path.isdir(output_path):
                os.makedirs(output_path)
            
            img.save(output_path+i+".jpg", format='JPEG')

Creating images: 100%|█████████████████████████████████████████████████████████████| 204/204 [00:56<00:00,  3.64item/s]
